In [3]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

### Global variables
PLACES_KEY = os.getenv("PLACES_KEY")
ROC = '43.1566,-77.6088'  # Rochester, NY coordinates
RADIUS = 15000  # Search within 15 km radius

In [11]:
### Creating python dictionary to hold all addresses and coordinates for food locations
""" Dictionary Format
dict {
    address:str -> (lat,long):tuple
    .
    .
    .
}

"""
GROCERY_LOC = dict() # 1
FAST_FOOD_LOC = dict() # 2
CORNERSTORE_LOC = dict() # 3
CONVENIENCE_LOC = dict() # 4
MINI_MART_LOC = dict() # 5
FARM_MARKET_LOC = dict() # 6
FOOD_BANK_LOC = dict() # 7
DRUGS_NLIQUOR_LOC = dict() # 8

DATA_POINTS = [GROCERY_LOC,FAST_FOOD_LOC,CORNERSTORE_LOC,CONVENIENCE_LOC,MINI_MART_LOC,FARM_MARKET_LOC,FOOD_BANK_LOC,DRUGS_NLIQUOR_LOC]


''

### Functions to process the Google Maps API call & process the json data

In [ ]:
from typing import Optional
def maps_query(query:str, location:str=ROC,radius:int=RADIUS,api_key:str=PLACES_KEY)->str:
    """_summary_

    Args:
        query (str): _description_
        location (str, optional): _description_. Defaults to ROC.
        radius (int, optional): _description_. Defaults to RADIUS.
        api_key (str, optional): _description_. Defaults to PLACES_KEY.

    Returns:
        str: search_url that will be sent to the google places api _description_
        
    """
    search_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={query}&key={api_key}"
    return search_url
def perform_search(input_url:str)->object:
    """_summary_

    Args:
        input_url (str): _description_

    Returns:
        object: json response from the google places api
    """
    response = requests.get(input_url)
    return response.json()
def process_places(json_data:dict,location_dict:dict,name_set:set)->set:
    for place in json_data['results']:
            
            

            ### Add name of location to do a set for a 2nd maps query to ensure all locations in ROC are accounted for
            name = place["name"]
            name_set.add(name)

            ########################################################################
            # If address is already in a location in DATA_POINTS continue to next entry in json
            address:str = place.get('vicinity','No address')
            address_break:bool = False # setting break condition: so that if the address appears in location dictionary the outer for loop goes to the next entry in json
           
            for location_dict in DATA_POINTS:
                if address in location_dict:
                    address_break = True
                    break
            if address_break:
                continue
            ########################################################################

            # If latitude and longitude is already a value in a location in DATA_POINTS continue to next entry in json
            lat:float = place['geometry']['location']['lat']
            lng:float = place['geometry']['location']['lng']
            coord_break:bool = False # setting break condition: so that if the coordinates appear in location dictionary the outer for loop goes to the next entry in json

            for location_dict in DATA_POINTS:
                if (lat,lng) in location_dict.values():
                    coord_break = True
            if coord_break:
                continue

            #########################################################################
            # If none of the above conditions are met, it's safe to add this address and pair of points to the location dictionary specified in the param
            # without fear of duplicate points
            location_dict[address] = (lat,lng)
    return name_set
        
def process_json(json_data:object,location_dict:dict, flag:bool)->None:
    """_summary_

    Args:
        json_data (object): _description_
        location_dict (dict): _description_
        flag (bool): if True, recursively check all the names of locations to ensure multiple different locations are account for
    """
    name_set:set = set()
    if flag == True:
        

        full_name_set = process_places(json_data,location_dict,name_set)
        
        ############################################################################
        # If there are multiple locations for a particular chain of grocery stores, food banks, liquor stores, etc.
        # check_names() ensures that all locations of the chain are accounted for in the ROC area
        check_names([full_name_set,location_dict])
    else:
        # If the flag is set to false, don't check the set of names that are captured
        # Just process the json data as normal
        process_places(json_data,location_dict,name_set)
        

def check_names(name_and_dict:list)->None:
    """_summary_
    Function verfifies that all locations of a particular chain of grocery stores, food banks, liquor stores, etc are accounted for.
    Args:
        name_and_dict (list): _description_
    """
    names:tuple = name_and_dict[0]
    location_dict:dict = name_and_dict[1]
    for name in names:
        search_url = maps_query(name)
        response_json = perform_search(search_url)
        process_json(response_json,location_dict,False)

def print_captured_locations(location_dict:dict)->None:
    """_summary_
    Prints the locations stored in the dictionary in clean easy to view format.
    Args:
        location_dict (dict): _description_
    """
    for key, value in location_dict.items():
        print(f"Location:{key} , coordinates{value[0],value[1]}")



##### Grocery Store Code

In [ ]:
GROCERY_QUERY = "grocery_or_supermarket"
grocery_search_url:str= maps_query(GROCERY_QUERY)
grocery_json = perform_search(grocery_search_url)
process_json(grocery_json,GROCERY_LOC,True)
print_captured_locations(GROCERY_LOC)

In [12]:



### Fast food query ###
ff_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type=&key={api_key}"

### Cornerstore 
cornerstore_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type=&key={api_key}"

### Convenience store  
convenience_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type=&key={api_key}"

### Mini mart

### Farmers Market

### Food Bank

### Liquor Stores ###
liquor_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type=&key={api_key}"

In [13]:
### Responses ###

ff_respone = requests.get(ff_url)
cornerstore_response = requests.get(cornerstore_url)
liquor__response = requests.get(liquor_url)


data = response.json()

print(data)
count: int = 0
for place in data['results']:
    name = place['name']
    address = place.get('vicinity', 'No address')
    lat = place['geometry']['location']['lat']
    lng = place['geometry']['location']['lng']
    print(f"{name} - {address} - Latitude: {lat}, Longitude: {lng}")
    count+=1
print()
print(count)

{'html_attributions': [], 'next_page_token': 'AWYs27x3jcGTM0oE3ujsrriDcUNP1JQ4TL3IfTccAaxpYUQK6dFR5nn6iQFdUw3UPI-_vnIon55sB57Huzg3qjYausg9-HB4JjST_xBve797IO4ue1uwnjn9nTkmUm-K02zMLO_48MwuUz8jslrQ5yvWP5YPHyld21vC2cztlSRq9VcRPWWZswKmHmk31kEULIHsFjWmveiq9mdAF3fU3WHeg6EK7Q82WNZLMIZbsEJI1GTsMEHGvc8054rUtMscm0ema_k435DhlQRp3dMW_fyfrq0Efwoj1YrEnfdrzc3QBAcSBVUL3GkogMGTmCi_JXvsG8ACeA7l46Emamr4FbLXQAwkN_yUP6Yq-n7aFnjzmi2wgpDmh3dyeLJ1_GQXa171ISkwOd-JgtIxMhCyLVmxYeQqIts4wKkzD6pVXG3upc6ASX_pmW52FPkEWuopYA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 43.1025054, 'lng': -77.5416475}, 'viewport': {'northeast': {'lat': 43.10394223029149, 'lng': -77.54015416970849}, 'southwest': {'lat': 43.10124426970849, 'lng': -77.54285213029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'icon_background_color': '#4B96F3', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/shoppingcart_pinlet', 'name': "Trader 

### Healthy ROC Grocer Program Partner Stores
[Healthy ROC Grocer Program](https://www.cityofrochester.gov/departments/mayors-office/healthy-roc-grocer-program)

* 999 Market	SE	709 S Clinton Ave
* Abundance Food Co-op	SE	571 South Ave
* Alba's Market	NW	1687 Dewey Ave
* Alice's Market	SE	1477 South Ave
* Bonga-Lo	SW	480 W Main St
* Bottomline Wholesale	NE	280 N Union St
* BSG Mini Mart	NW	480 Dewey Ave
* Convenient Market and Deli	NE	1430 N Clinton Ave
* D&L Tropical Groceries	SW	1005 Genesee St
* Dybowski Authentic Polish Market	NE	1325 Hudson Ave
* Dream Town Deli 	SW	275 East Ave
* Empire Meats	NW	1356 Lyell Ave
* Empire Meats	SE	1337 N Goodman St
* Empire Meats	SW	626 W Main St
* Gorkha Store	SW	537 State St
* Grape and Orange Mini Market	NW	111 Orange St
* Grocery Bazaar	NW	639 Lake Ave
* Halal Market & Meats Inc.	NE	311 E Ridge Rd
* Highland Market 	SE	830 South Ave 
* Homegrown and Handmade Market	SE	986 Monroe Ave
* New Hikari	SW	1667 Mt Hope Ave
* Olindo's Cash and Carry	NW	1510 Lyell Ave
* Plymouth Express Deli	SW	743 S Plymouth Ave
* Public Meat & Grocery 	NE	600 Hudson Ave
* Rochester Public Market	NE	280 North Union St
* Skip's on the Ridge	NW	640 W Ridge Rd

In [ ]:
### Reverse Geo-coding Healthy ROC Grocer Locations and ensuring all locations are included 